In [ ]:

!pip install xformers
!pip install transformers[torch]
!pip install datasets
!pip install  torchtext


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━ 56.5/124.2 MB 2.3 MB/s eta 0:00:30

In [ ]:

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
import os
import transformers
from transformers import AdamW, get_linear_schedule_with_warmup, AutoModelForSequenceClassification, AutoModel, set_seed,  GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, GPT2Model,  GPT2LMHeadModel, pipeline, Trainer, TrainingArguments,GPT2Config
import torch
from torch import nn
from torch.utils.data import Dataset as D1, DataLoader
from tqdm import tqdm
from datasets import Dataset, load_dataset
from collections import defaultdict
import time

# Misc.
import warnings
warnings.filterwarnings('ignore')

In [ ]:
batch_size = 6
MAX_LEN = 30
RANDOM_SEED = 16
MODEL_NAME = "gpt2"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
EPOCHS = 1
token_size = 50257


In [ ]:
def MODEL(n_layer):

    model = GPT2LMHeadModel.from_pretrained('gpt2',n_layer=n_layer)

    return model

In [ ]:
data_set = 'bookcorpus'

In [ ]:
df = pd.read_excel("webtext.xlsx")

In [ ]:
len(df)

In [ ]:
df = df.head(400)

In [ ]:
len(df)

In [ ]:

max_ = float("-inf")
for i in df["text"]:

  max_ = max(len(i),max_)

print(max_)

CUTOFF_LEN = max_


In [ ]:
a = torch.randn(batch_size,CUTOFF_LEN,token_size)

def give_input_shape(a):

    c1 = nn.Conv1d(CUTOFF_LEN, 32, 3, stride=2)
    c2 = nn.Conv1d(32, 32, 3, stride=2)
    p1 = nn.MaxPool2d(3, stride=2)
    f = nn.Flatten()

    a = c1(a)
    a = c2(a)
    a = p1(a)
    a = f(a)

    return a.shape[1]

input_dim = give_input_shape(a)

In [ ]:
input_dim

In [ ]:
def Discriminator():

    D = nn.Sequential(
    nn.Conv1d(CUTOFF_LEN, 32, 3, stride=2),
    nn.Conv1d(32, 32, 3, stride=2),
    nn.MaxPool2d(3, stride=2),
    nn.Flatten(),
    nn.Linear(input_dim, 256),
    nn.LeakyReLU(0.2),
    nn.Linear(256, 1),
    nn.Sigmoid())

    return D

In [ ]:
train,val = train_test_split(df,random_state = 42,test_size=0.2)

In [ ]:
len(train)

320

In [ ]:
len(val)

80

In [ ]:
df.head(10)

,Unnamed: 0,text
0,0,"usually , he would be tearing around the livin..."
1,1,but just one look at a minion sent him practic...
2,2,that had been megan 's plan when she got him d...
3,3,"he 'd seen the movie almost by mistake , consi..."
4,4,she liked to think being surrounded by adults ...
5,5,`` are n't you being a good boy ? ''
6,6,she said .
7,7,mason barely acknowledged her .
8,8,"instead , his baby blues remained focused on t..."
9,9,"since the movie was almost over , megan knew s..."


In [ ]:
data_train = Dataset.from_dict({"text" : list(train["text"]) })
data_val = Dataset.from_dict({"text" : list(val["text"]) })

In [ ]:
data_train



Dataset({
    features: ['text'],
    num_rows: 320
})

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

tokenizer.pad_token = tokenizer.eos_token

In [ ]:
def tokenize(prompt):
    # there's probably a way to do this with the tokenizer settings
    # but again, gotta move fast
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=CUTOFF_LEN,
        padding='max_length',
        return_tensors=None,
    )

    result["labels"] = result["input_ids"].copy()

    return result


def generate_and_tokenize_prompt(data_point):

    tokenized_full_prompt = tokenize(data_point["text"])

    return tokenized_full_prompt

def create_data_pt(data,batch_size = batch_size):

    data.set_format(type='torch', columns=['text', 'input_ids', 'attention_mask', 'labels'])
    data = torch.utils.data.DataLoader(data, batch_size=batch_size)
    return data

In [ ]:
train_d = data_train.shuffle().map(generate_and_tokenize_prompt)
val_d = data_val.shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/320 [00:00<?, ? examples/s]

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

In [ ]:
train_d2 =  create_data_pt(train_d)
val_d2 =  create_data_pt(val_d)

In [ ]:
train_d2


In [ ]:
t_n_layer = 6
s_n_layer = 3

In [ ]:
M1 = MODEL(t_n_layer)
M2 = MODEL(s_n_layer)
D = Discriminator()


optimizer = AdamW(M2.parameters(), lr=2e-5, correct_bias=False)
D_optimizer = AdamW(D.parameters(), lr=2e-4, correct_bias=False)


total_steps = len(train_d2) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)


Some weights of the model checkpoint at gpt2 were not used when initializing GPT2LMHeadModel: ['h.6.attn.c_proj.weight', 'h.10.attn.c_proj.weight', 'h.6.ln_1.weight', 'h.9.ln_2.weight', 'h.7.mlp.c_proj.weight', 'h.11.mlp.c_proj.bias', 'h.7.attn.c_attn.weight', 'h.8.ln_1.weight', 'h.11.mlp.c_fc.bias', 'h.8.attn.c_attn.bias', 'h.11.ln_1.weight', 'h.8.mlp.c_fc.bias', 'h.9.ln_2.bias', 'h.8.attn.c_proj.weight', 'h.9.mlp.c_proj.weight', 'h.6.ln_2.bias', 'h.10.attn.c_attn.weight', 'h.6.mlp.c_fc.bias', 'h.6.ln_2.weight', 'h.11.ln_1.bias', 'h.10.mlp.c_proj.weight', 'h.11.ln_2.weight', 'h.7.mlp.c_fc.weight', 'h.11.mlp.c_fc.weight', 'h.9.ln_1.weight', 'h.10.ln_2.bias', 'h.7.attn.c_proj.bias', 'h.8.mlp.c_proj.bias', 'h.9.mlp.c_proj.bias', 'h.8.attn.c_attn.weight', 'h.8.attn.c_proj.bias', 'h.6.attn.bias', 'h.7.mlp.c_fc.bias', 'h.6.mlp.c_proj.bias', 'h.11.attn.bias', 'h.11.ln_2.bias', 'h.9.attn.c_attn.weight', 'h.7.mlp.c_proj.bias', 'h.10.ln_1.weight', 'h.10.ln_2.weight', 'h.7.ln_1.bias', 'h.8.ln_2.

In [ ]:

cross_loss = nn.CrossEntropyLoss()
cosine_loss = nn.CosineEmbeddingLoss()
criterion = nn.BCELoss()



def discriminator_loss(output1,output2,D):


    batch_size = len(output1)

    real_labels = torch.ones(batch_size, 1).to(DEVICE)
    fake_labels = torch.zeros(batch_size, 1).to(DEVICE)


    outputs1 = D(output1)
    d_loss_real = criterion(outputs1, real_labels)



    outputs2 = D(output2)
    d_loss_fake = criterion(outputs2, fake_labels)


    d_loss = d_loss_real + d_loss_fake


    return d_loss




def student_gan_loss(student_logits,D):



    labels = torch.ones(len(student_logits), 1).to(DEVICE)
    g_loss = criterion(D(student_logits), labels)

    return g_loss





def kd_loss(teacher_logits,student_logits,T=2):

    soft_targets = nn.functional.softmax(teacher_logits / T, dim=-1)
    soft_prob = nn.functional.softmax(student_logits / T, dim=-1)    #log_softmax

    entropy_loss_val = cross_loss(soft_prob,soft_targets)

    return entropy_loss_val

def cosine_sim_loss(teacher_logits,student_logits,T= 2):


    cosine_loss_val = 0
    n = len(teacher_logits)

    for i,j in zip(teacher_logits,student_logits):

        soft_targets = nn.functional.softmax(i / T, dim=-1)
        soft_prob = nn.functional.softmax(i / T, dim=-1)           #log_softmax

        y = torch.ones(soft_prob.shape[0])

        y = y.to(DEVICE)

        cosine_loss_val += cosine_loss(soft_prob,soft_targets,y)

    return cosine_loss_val /n



def train_epoch(M1,M2,D,a = 0.25,b = 0.25 ,c = 0.25 ,d = 0.25):



    M1 = M1.eval()  # Teacher set to evaluation mode
    M2 = M2.train()
    total_train_loss = 0


    steps = 0


    for batch in tqdm(train_d2):


        b_input_ids = batch["input_ids"].to(DEVICE)
        b_masks = batch["attention_mask"].to(DEVICE)
        b_labels = batch["labels"].to(DEVICE)

        optimizer.zero_grad()
        # D_optimizer.zero_grad()

        with torch.no_grad():

            outputs1 = M1(b_input_ids,
                          labels=b_labels,
                          attention_mask = b_masks)


        outputs2 = M2(b_input_ids,
                          labels=b_labels,
                          attention_mask = b_masks)


        teacher_logits = outputs1[1]
        student_logits = outputs2[1]



        stu_loss = student_gan_loss(student_logits,D)


        entropy_loss_val =  kd_loss(teacher_logits,student_logits)



        auto_reg_loss = outputs2[0]


        cosine_loss_val = cosine_sim_loss(teacher_logits,student_logits)



        loss = (a*entropy_loss_val + b*auto_reg_loss + c*cosine_loss_val + d*stu_loss)

        total_train_loss += loss.item()

        steps += 1

        loss.backward()

        optimizer.step()
        scheduler.step()



    return total_train_loss/steps



def train_discriminator(M1,M2,D,T=2):


    M1 = M1.eval()
    M2 = M2.eval()
    D = D.train()


    steps = 0
    total_dis_loss = 0

    for batch in tqdm(train_d2):

        b_input_ids = batch["input_ids"].to(DEVICE)
        b_masks = batch["attention_mask"].to(DEVICE)
        b_labels = batch["labels"].to(DEVICE)

        D_optimizer.zero_grad()


        with torch.no_grad():

            outputs1 = M1(b_input_ids,
                        labels=b_labels,
                        attention_mask = b_masks)

            outputs2 = M2(b_input_ids,
                        labels=b_labels,
                        attention_mask = b_masks)


        teacher_logits = outputs1[1]
        student_logits = outputs2[1]

        teacher_logits = nn.functional.softmax(teacher_logits / T, dim=-1)
        student_logits = nn.functional.softmax(student_logits / T, dim=-1)


        batch_size = len(student_logits)

        real_labels = torch.ones(batch_size, 1).to(DEVICE)
        fake_labels = torch.zeros(batch_size, 1).to(DEVICE)


        o1 = D(teacher_logits)
        d_loss_real = criterion(o1, real_labels)



        o2 = D(student_logits)
        d_loss_fake = criterion(o2, fake_labels)


        d_loss = d_loss_real + d_loss_fake


        total_dis_loss += d_loss.item()

        steps += 1




        d_loss.backward()
        D_optimizer.step()

    return total_dis_loss/steps


def eval_model(M1,M2,D,a = 0.25,b = 0.25 ,c = 0.25 ,d = 0.25):



    M1 = M1.eval()  # Teacher set to evaluation mode
    M2 = M2.train()
    total_val_loss = 0
    steps = 0



    for batch in tqdm(val_d2):


        b_input_ids = batch["input_ids"].to(DEVICE)
        b_masks = batch["attention_mask"].to(DEVICE)
        b_labels = batch["labels"].to(DEVICE)


        with torch.no_grad():

            outputs1 = M1(b_input_ids,
                          labels=b_labels,
                          attention_mask = b_masks)



            outputs2 = M2(b_input_ids,
                            labels=b_labels,
                            attention_mask = b_masks)


            teacher_logits = outputs1[1]
            student_logits = outputs2[1]




            stu_loss = student_gan_loss(student_logits,D)


            entropy_loss_val =  kd_loss(teacher_logits,student_logits)

            auto_reg_loss = outputs2[0]

            cosine_loss_val = cosine_sim_loss(teacher_logits,student_logits)

            loss = (a*entropy_loss_val + b*auto_reg_loss + c*cosine_loss_val + d*stu_loss)



            total_val_loss  += loss.item()

            steps += 1



    return total_val_loss/steps

def train(M1,M2,D,EPOCHS,verbose = True):

    M1 = M1.to(DEVICE)
    M2 = M2.to(DEVICE)
    D = D.to(DEVICE)


    if(verbose == True):

        print()
        print()
        print()
        print(M1)
        print()
        print("<->"*100)
        print()
        print(M2)
        print()
        print("<->"*100)
        print()
        print(D)
        print()
        print()
        print()


    # model = model.to(DEVICE)

    tt = 0
    d = defaultdict(list)



    for epoch in range(EPOCHS):

        print()
        print(f'Epoch {epoch + 1}/{EPOCHS}')
        print('-' * 20)


        s = time.time()
        train_loss = train_epoch(M1,M2,D,)
        e = time.time()

        print("Time per epoch for training", (e-s))


        tt += (e-s)


        val_loss = eval_model(M1,M2,D,)

        dis_loss = train_discriminator(M1,M2,D,)

        print()
        print()
        print("-"*100)
        print()
        print(f' Train_loss {train_loss:.5f}  <->  Val_loss {val_loss:.5f} <-> Discriminator Loss {dis_loss: .5f}')
        print()
        print("-"*100)
        print()
        print()


        d["Train_loss"].append(train_loss)
        d["Val_loss"].append(val_loss)
        d["Discriminator_Loss"].append(dis_loss)

    print()
    print()
    print("*"*100)
    print()
    print()
    print()
    print("Total time taken: ", tt)
    print("Average time per epoch: ", tt/EPOCHS)
    print()
    print()
    print()
    print("*"*100)
    print()
    print()

    return d


In [ ]:



if(True):



    # try:
    print()
    print("#"*100)
    print()
    print()
    print("#"*100)
    print()
    print()
    print("The Actual method")
    print("num_teacher_decoders: ", t_n_layer)
    print("num_student_decoders: ", s_n_layer)
    print()
    print()

    history1 = train(M1,M2,D,4)
    d = pd.DataFrame(history1)

    print()
    print("*"*100)
    print()

    print()
    print("*"*100)
    print()

    print()
    print(d)
    print()




    print("#"*100)
    print()
    print()
    print()
    print("#"*100)
    print()

    del M1
    # del M2
    torch.cuda.empty_cache()

    # except Exception as e:

        # print(e)





####################################################################################################


####################################################################################################


The Actual method
num_teacher_decoders:  6
num_student_decoders:  3





GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)

100%|██████████| 54/54 [00:24<00:00,  2.21it/s]


Time per epoch for training 24.445485591888428


100%|██████████| 54/54 [00:13<00:00,  4.01it/s]




----------------------------------------------------------------------------------------------------

 Train_loss 0.20622  <->  Val_loss 0.10783 <-> Discriminator Loss  1.59758

----------------------------------------------------------------------------------------------------



Epoch 2/4
--------------------


...



----------------------------------------------------------------------------------------------------

 Train_loss 0.10144  <->  Val_loss 0.10768 <-> Discriminator Loss  0.04917

----------------------------------------------------------------------------------------------------



Epoch 4/4
--------------------


100%|██████████| 54/54 [00:24<00:00,  2.17it/s]


Time per epoch for training 24.848533868789673


100%|██████████| 54/54 [00:12<00:00,  4.31it/s]



----------------------------------------------------------------------------------------------------

 Train_loss 0.10176  <->  Val_loss 0.10761 <-> Discriminator Loss  0.03422

----------------------------------------------------------------------------------------------------




****************************************************************************************************



Total time taken:  98.88733172416687
Average time per epoch:  24.721832931041718



****************************************************************************************************



****************************************************************************************************


****************************************************************************************************


   Train_loss  Val_loss  Discriminator_Loss
0    0.206216  0.107827            1.597581
1    0.101820  0.107920            0.447581
2    0.101436  0.107682            0.049168
3    0.101759  0.107606            0.0342

In [ ]:
del M2

In [ ]:
del M1

In [ ]:
m = MODEL(4)
m = m.to(DEVICE)

Some weights of the model checkpoint at gpt2 were not used when initializing GPT2LMHeadModel: ['h.7.mlp.c_fc.bias', 'h.9.attn.c_attn.weight', 'h.6.attn.c_attn.weight', 'h.6.attn.bias', 'h.9.ln_2.bias', 'h.11.attn.c_proj.weight', 'h.9.attn.bias', 'h.4.mlp.c_proj.weight', 'h.5.attn.c_attn.weight', 'h.10.mlp.c_fc.weight', 'h.11.attn.c_attn.weight', 'h.6.attn.c_proj.weight', 'h.7.ln_2.bias', 'h.9.mlp.c_fc.bias', 'h.7.ln_1.weight', 'h.7.mlp.c_proj.weight', 'h.8.mlp.c_fc.bias', 'h.4.attn.c_attn.bias', 'h.5.ln_1.bias', 'h.5.mlp.c_fc.weight', 'h.9.mlp.c_fc.weight', 'h.10.mlp.c_fc.bias', 'h.10.attn.c_attn.weight', 'h.7.attn.c_proj.weight', 'h.8.attn.c_proj.bias', 'h.4.ln_2.weight', 'h.6.attn.c_attn.bias', 'h.4.mlp.c_fc.weight', 'h.6.ln_2.bias', 'h.7.ln_1.bias', 'h.7.ln_2.weight', 'h.9.attn.c_attn.bias', 'h.6.mlp.c_proj.weight', 'h.8.attn.c_attn.bias', 'h.9.attn.c_proj.weight', 'h.11.ln_1.bias', 'h.6.mlp.c_fc.bias', 'h.6.ln_1.bias', 'h.9.mlp.c_proj.bias', 'h.10.mlp.c_proj.weight', 'h.4.mlp.c_pro

In [ ]:
D = Discriminator()

In [ ]:
D = D.to(DEVICE)

In [ ]:
D_optimizer = AdamW(D.parameters(), lr=2e-5, correct_bias=False)

In [ ]:
M1 = MODEL(4)
M1 = M1.to(DEVICE)

M2 = MODEL(4)
M2 = M2.to(DEVICE)

Some weights of the model checkpoint at gpt2 were not used when initializing GPT2LMHeadModel: ['h.8.ln_1.bias', 'h.11.attn.c_proj.weight', 'h.4.attn.c_attn.bias', 'h.8.ln_1.weight', 'h.8.mlp.c_fc.bias', 'h.8.mlp.c_proj.bias', 'h.9.attn.c_attn.bias', 'h.9.attn.c_proj.weight', 'h.11.attn.c_attn.weight', 'h.6.mlp.c_fc.bias', 'h.11.mlp.c_fc.bias', 'h.9.mlp.c_proj.weight', 'h.4.mlp.c_proj.bias', 'h.10.mlp.c_proj.weight', 'h.6.attn.c_proj.weight', 'h.7.attn.c_proj.bias', 'h.9.attn.bias', 'h.8.ln_2.bias', 'h.5.attn.c_proj.weight', 'h.6.mlp.c_fc.weight', 'h.5.ln_2.bias', 'h.11.mlp.c_fc.weight', 'h.5.mlp.c_fc.bias', 'h.4.attn.c_proj.weight', 'h.8.attn.c_proj.weight', 'h.6.ln_2.bias', 'h.7.attn.bias', 'h.11.attn.bias', 'h.7.ln_1.bias', 'h.7.attn.c_proj.weight', 'h.6.attn.c_proj.bias', 'h.4.ln_2.bias', 'h.5.mlp.c_proj.bias', 'h.5.ln_1.weight', 'h.9.ln_2.bias', 'h.9.mlp.c_fc.bias', 'h.11.ln_1.bias', 'h.7.ln_2.bias', 'h.7.attn.c_attn.weight', 'h.10.mlp.c_fc.bias', 'h.6.mlp.c_proj.bias', 'h.5.attn.c

Some weights of the model checkpoint at gpt2 were not used when initializing GPT2LMHeadModel: ['h.8.ln_1.bias', 'h.11.attn.c_proj.weight', 'h.4.attn.c_attn.bias', 'h.8.ln_1.weight', 'h.8.mlp.c_fc.bias', 'h.8.mlp.c_proj.bias', 'h.9.attn.c_attn.bias', 'h.9.attn.c_proj.weight', 'h.11.attn.c_attn.weight', 'h.6.mlp.c_fc.bias', 'h.11.mlp.c_fc.bias', 'h.9.mlp.c_proj.weight', 'h.4.mlp.c_proj.bias', 'h.10.mlp.c_proj.weight', 'h.6.attn.c_proj.weight', 'h.7.attn.c_proj.bias', 'h.9.attn.bias', 'h.8.ln_2.bias', 'h.5.attn.c_proj.weight', 'h.6.mlp.c_fc.weight', 'h.5.ln_2.bias', 'h.11.mlp.c_fc.weight', 'h.5.mlp.c_fc.bias', 'h.4.attn.c_proj.weight', 'h.8.attn.c_proj.weight', 'h.6.ln_2.bias', 'h.7.attn.bias', 'h.11.attn.bias', 'h.7.ln_1.bias', 'h.7.attn.c_proj.weight', 'h.6.attn.c_proj.bias', 'h.4.ln_2.bias', 'h.5.mlp.c_proj.bias', 'h.5.ln_1.weight', 'h.9.ln_2.bias', 'h.9.mlp.c_fc.bias', 'h.11.ln_1.bias', 'h.7.ln_2.bias', 'h.7.attn.c_attn.weight', 'h.10.mlp.c_fc.bias', 'h.6.mlp.c_proj.bias', 'h.5.attn.c

In [ ]:
optimizer = AdamW(M2.parameters(), lr=2e-5, correct_bias=False)

In [ ]:
cross_loss = nn.CrossEntropyLoss()
cosine_loss = nn.CosineEmbeddingLoss()
criterion = nn.BCELoss()




# def discriminator_loss(output1,output2,D):




#     batch_size = len(output1)

#     real_labels = torch.ones(batch_size, 1).to(DEVICE)
#     fake_labels = torch.zeros(batch_size, 1).to(DEVICE)


#     outputs1 = D(output1)
#     d_loss_real = criterion(outputs1, real_labels)



#     outputs2 = D(output2)
#     d_loss_fake = criterion(outputs2, fake_labels)


#     d_loss = d_loss_real + d_loss_fake

#     D_optimizer.zero_grad()
#     dis_loss.backward(retain_graph=True)
#     D_optimizer.step()


#     return d_loss




def student_gan_loss(student_logits,D):

    labels = torch.ones(batch_size, 1).to(DEVICE)
    g_loss = criterion(D(student_logits), labels)

    return g_loss





def kd_loss(teacher_logits,student_logits,T=2):

    soft_targets = nn.functional.softmax(teacher_logits / T, dim=-1)
    soft_prob = nn.functional.log_softmax(student_logits / T, dim=-1)

    entropy_loss_val = cross_loss(soft_prob,soft_targets)

    return entropy_loss_val

def cosine_sim_loss(teacher_logits,student_logits,T= 2):


    cosine_loss_val = 0
    n = len(teacher_logits)

    for i,j in zip(teacher_logits,student_logits):

        soft_targets = nn.functional.softmax(i / T, dim=-1)
        soft_prob = nn.functional.log_softmax(i / T, dim=-1)

        y = torch.ones(soft_prob.shape[0])

        y = y.to(DEVICE)

        cosine_loss_val += cosine_loss(soft_prob,soft_targets,y)

    return cosine_loss_val /n


steps = 0
total_train_loss = 0



for batch in tqdm(train_d2):



    b_input_ids = batch["input_ids"].to(DEVICE)
    b_masks = batch["attention_mask"].to(DEVICE)
    b_labels = batch["labels"].to(DEVICE)

    optimizer.zero_grad()



    with torch.no_grad():

        outputs1 = M1(b_input_ids,
                    labels=b_labels,
                    attention_mask = b_masks)


    outputs2 = M2(b_input_ids,
                    labels=b_labels,
                    attention_mask = b_masks)


    teacher_logits = outputs1[1]
    student_logits = outputs2[1]



    # soft_targets = nn.functional.softmax(teacher_logits / 2, dim=-1)
    # soft_prob = nn.functional.log_softmax(student_logits / 2, dim=-1)


    # dis_loss = discriminator_loss(teacher_logits,student_logits,D)


    stu_loss = student_gan_loss(student_logits,D)


    entropy_loss_val =  kd_loss(teacher_logits,student_logits)

    auto_reg_loss = outputs2[0]

    cosine_loss_val = cosine_sim_loss(teacher_logits,student_logits)

    loss = (entropy_loss_val + auto_reg_loss + cosine_loss_val +stu_loss)/4


    total_train_loss += loss.item()


    steps += 1

    loss.backward()


    optimizer.step()


    t1 = total_train_loss/steps




    if(steps == 3):

        break



steps = 0
total_dis_loss = 0

for batch in tqdm(train_d2):

    b_input_ids = batch["input_ids"].to(DEVICE)
    b_masks = batch["attention_mask"].to(DEVICE)
    b_labels = batch["labels"].to(DEVICE)

    D_optimizer.zero_grad()


    with torch.no_grad():

        outputs1 = M1(b_input_ids,
                    labels=b_labels,
                    attention_mask = b_masks)

        outputs2 = M2(b_input_ids,
                    labels=b_labels,
                    attention_mask = b_masks)


    teacher_logits = outputs1[1]
    student_logits = outputs2[1]


    batch_size = len(student_logits)

    real_labels = torch.ones(batch_size, 1).to(DEVICE)
    fake_labels = torch.zeros(batch_size, 1).to(DEVICE)


    o1 = D(teacher_logits)
    d_loss_real = criterion(o1, real_labels)



    o2 = D(student_logits)
    d_loss_fake = criterion(o2, fake_labels)


    d_loss = d_loss_real + d_loss_fake


    total_dis_loss += d_loss.item()

    steps += 1




    d_loss.backward()
    D_optimizer.step()

    if(steps == 3):

        break














 29%|██▊       | 2/7 [00:00<00:01,  3.44it/s]


In [ ]:
soft_targets = nn.functional.softmax(teacher_logits / 2, dim=-1)
soft_prob = nn.functional.log_softmax(student_logits / 2, dim=-1)

In [ ]:
soft_targets

tensor([[[1.9224e-04, 2.0787e-04, 6.4163e-05,  ..., 9.1992e-06,
          1.0541e-05, 2.1611e-04],
         [2.0253e-04, 1.8524e-04, 1.4469e-06,  ..., 3.1037e-06,
          7.9252e-06, 1.0457e-05],
         [1.2642e-04, 5.6397e-05, 4.4338e-06,  ..., 2.9769e-08,
          2.5920e-07, 4.2691e-06],
         ...,
         [3.0507e-05, 2.9509e-03, 4.6985e-04,  ..., 2.3781e-07,
          4.8545e-08, 2.7130e-03],
         [3.0058e-05, 2.9332e-03, 4.6921e-04,  ..., 2.3252e-07,
          4.7802e-08, 2.7119e-03],
         [3.0793e-05, 2.9737e-03, 4.7422e-04,  ..., 2.3772e-07,
          4.8785e-08, 2.7517e-03]],

        [[1.8541e-04, 1.9811e-04, 6.0498e-05,  ..., 9.3748e-06,
          1.0803e-05, 2.1058e-04],
         [3.2558e-04, 2.1044e-04, 5.1318e-05,  ..., 5.3115e-08,
          3.7193e-07, 1.6476e-04],
         [2.8958e-05, 1.2620e-05, 1.0072e-06,  ..., 2.6618e-06,
          1.0340e-05, 3.2915e-06],
         ...,
         [1.9630e-05, 1.6429e-03, 3.4465e-04,  ..., 2.0159e-07,
          9.001

In [ ]:
soft_prob

tensor([[[ -8.5407,  -8.4940,  -9.6640,  ..., -11.6055, -11.4552,  -8.3311],
         [ -7.5650,  -8.8051, -12.9451,  ..., -13.6839, -11.9905,  -7.1827],
         [ -6.9696,  -8.8572, -10.9832,  ..., -16.4497, -14.7009,  -8.2570],
         ...,
         [-10.1908, -10.5707, -11.3095,  ..., -19.6375, -19.3610,  -0.0764],
         [-10.2026, -10.5620, -11.3012,  ..., -19.6264, -19.3502,  -0.0755],
         [-10.1909, -10.5402, -11.2874,  ..., -19.6098, -19.3348,  -0.0767]],

        [[ -8.5774,  -8.5349,  -9.7188,  ..., -11.5896, -11.4344,  -8.3653],
         [ -6.8229,  -8.3580,  -9.8481,  ..., -16.6740, -14.4083,  -4.6330],
         [ -9.1092, -11.2707, -13.6073,  ..., -13.2193, -12.4118,  -8.5256],
         ...,
         [-11.6779, -12.6136, -13.3653,  ..., -19.8713, -19.4888,  -0.0343],
         [-11.7038, -12.6159, -13.3590,  ..., -19.8762, -19.5141,  -0.0335],
         [-11.6990, -12.5992, -13.3472,  ..., -19.8447, -19.4882,  -0.0339]],

        [[ -8.5595,  -8.5386,  -9.7662,  ...

In [ ]:
for batch in tqdm(train_d2):


    b_input_ids = batch["input_ids"].to(DEVICE)
    b_masks = batch["attention_mask"].to(DEVICE)
    b_labels = batch["labels"].to(DEVICE)

    optimizer.zero_grad()
    # D_optimizer.zero_grad()

    with torch.no_grad():

        outputs1 = M1(b_input_ids,
                        labels=b_labels,
                        attention_mask = b_masks)


    outputs2 = M2(b_input_ids,
                        labels=b_labels,
                        attention_mask = b_masks)


    teacher_logits = outputs1[1]
    student_logits = outputs2[1]



    # soft_targets = nn.functional.softmax(teacher_logits / 2, dim=-1)
    # soft_prob = nn.functional.log_softmax(student_logits / 2, dim=-1)


    # dis_loss = discriminator_loss(teacher_logits,student_logits,D)

    stu_loss = student_gan_loss(student_logits,D)


    entropy_loss_val =  kd_loss(teacher_logits,student_logits)

    auto_reg_loss = outputs2[0]

    cosine_loss_val = cosine_sim_loss(teacher_logits,student_logits)

    loss = (entropy_loss_val + auto_reg_loss + cosine_loss_val +stu_loss)/4


    total_train_loss += loss.item()
    # total_dis_loss += dis_loss.item()

    steps += 1

    loss.backward()
    # dis_loss.backward()

    # D_optimizer.step()
    optimizer.step()

In [ ]:
total_loss

tensor(2.0383, device='cuda:0', grad_fn=<DivBackward0>)

In [ ]:
entropy_loss_val

tensor(0.0204, device='cuda:0', grad_fn=<DivBackward1>)

In [ ]:
cosine_loss_val

tensor(1.0559, device='cuda:0')

In [ ]:
auto_reg_loss

tensor(6.6020, device='cuda:0', grad_fn=<NllLossBackward0>)

In [ ]:
stu_loss

tensor(0.4750, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)

In [ ]:
len(outputs2[1])

6

In [ ]:
total_loss

tensor(2.5595, device='cuda:0', grad_fn=<DivBackward0>)

In [ ]:
len(teacher_logits)

6

In [ ]:
entropy_loss_val

tensor(7.7493, device='cuda:0', grad_fn=<DivBackward1>)

In [ ]:
teacher_logits.shape

torch.Size([205, 50257])

In [ ]:
entropy_loss_val

tensor(7.7493, device='cuda:0', grad_fn=<DivBackward1>)

In [ ]:
teacher_logits.shape

torch.Size([205, 50257])

In [ ]:
outputs1[1].shape

torch.Size([6, 205, 50257])

In [ ]:
D(outputs1[1][0])

RuntimeError: ignored

In [ ]:
soft_targets.shape

torch.Size([205, 50257])

In [ ]:
outputs[1].shape

torch.Size([6, 205, 50257])

In [ ]:

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.input_shape = (205, 50257)

        # Flatten the input
        self.flatten = nn.Flatten()

        # Define the dense layers
        self.fc1 = nn.Linear(205 * 50257, 1024)
        self.fc2 = nn.Linear(1024, 1024)
        self.fc3 = nn.Linear(1024, 1024)
        self.fc4 = nn.Linear(1024, 512)
        # Define activation functions
        self.relu = nn.LeakyReLU(0.2)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.fc4(x)
        x = self.sigmoid(x)  # Sigmoid activation for binary classification

        return x



In [ ]:
D = Discriminator()

In [ ]:
50257*205

In [ ]:
1000000000
10302685

In [ ]:
image_size = 10302685
hidden_size = 512

In [ ]:
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid())

In [ ]:
(224*224*3) / 10302685

In [ ]:
data = torch.randn(100, 1000)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=100)
reduced_data = pca.fit_transform(data)

In [ ]:
discriminator

Discriminator(
  (conv_layer): Conv1d(3, 1, kernel_size=(3,), stride=(2,))
  (output_layer): Sigmoid()
)

In [ ]:
x = torch.randn((6, 205, 50257))

In [ ]:
outputs[1].shape

torch.Size([6, 205, 50257])

In [ ]:
x.shape

torch.Size([10, 3, 100])

In [ ]:
output = discriminator(x)

RuntimeError: ignored

In [ ]:
m = nn.Conv1d(205, 32, 3, stride=2)
input = torch.randn(6, 205, 50257)
output = m(input)

In [ ]:
output.shape

torch.Size([6, 32, 25128])

In [ ]:
# pool of square window of size=3, stride=2
# m = nn.MaxPool2d(3, stride=2)
# # pool of non-square window


In [ ]:
output.shape

torch.Size([20, 16, 24, 31])

In [ ]:
# pool of square window of size=3, stride=2
m = nn.MaxPool2d(3, stride=2)

input = torch.randn(6, 205, 50257)
output = m(input)

In [ ]:
output.shape

torch.Size([6, 102, 25128])

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, in_channels=1, out_channels=32, kernel_size=3 ):
        super(Discriminator, self).__init__()

        # Convolutional layer
        self.conv1d = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3)

        # Max pooling layer
        self.maxpool = nn.MaxPool1d(kernel_size=2)

        # Fully connected layers
        self.fc1 = nn.Linear(64 * (input_length // 2 - 2), 128)  # Adjust input size accordingly
        self.fc2 = nn.Linear(128, 1)  # Output layer for binary classification

        # Activation functions
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Apply convolutional layer
        x = self.conv1d(x)
        x = self.relu(x)

        # Apply max pooling
        x = self.maxpool(x)

        # Flatten the output
        x = x.view(x.size(0), -1)

        # Apply fully connected layers
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)

        return x

In [ ]:
6, 205, 50257

In [ ]:
batch_size = 6
in_channels = 205
input_length = 50257
input_data = torch.randn(batch_size, in_channels, input_length)

In [ ]:
discriminator = Discriminator(in_channels=205)

TypeError: ignored

In [ ]:

input = torch.randn(6, 205, 50257)





In [ ]:
D = Discriminator()

In [ ]:
d = D(input)

In [ ]:
d

tensor([[0.4796],
        [0.4800],
        [0.4738],
        [0.4835],
        [0.4920],
        [0.4925]], grad_fn=<SigmoidBackward0>)

In [ ]:
cov = nn.Conv1d(205, 32, 3, stride=2)
cov2 = nn.Conv1d(32, 32, 3, stride=2)
pool = nn.MaxPool2d(3, stride=2)
flat = nn.Flatten()

In [ ]:
o1 = cov(input)

In [ ]:
o1.shape

torch.Size([6, 32, 25128])

In [ ]:
o1 = cov2(o1)

In [ ]:
o1.shape

torch.Size([6, 32, 12563])

In [ ]:
p = pool(o1)

In [ ]:
p.shape

torch.Size([6, 15, 6281])

In [ ]:
f = flat(p)

In [ ]:
f.shape

torch.Size([6, 94215])

In [ ]:
205*50257

10302685

In [ ]:
1130670/10302685

0.10974517807736527

In [ ]:
class Discriminator(nn.Module):
    def __init__(self:
        super(Discriminator, self).__init__()

        # 1D convolutional layer
        self.conv1d = nn.Conv1d(in_channels=205, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2d = nn.Conv1d(in_channels=32 out_channels=32, kernel_size=3, stride=1, padding=1)
        self.flatten = nn.Flatten()
        self.

        # Max pooling layer
        self.maxpool = nn.MaxPool1d(kernel_size=2)


        self.fc1 = nn.Linear(64, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = torch.relu(self.conv1d(x))
        x = tor
        x = self.maxpool(x)

        # Flatten the output for fully connected layers
        x = x.view(x.size(0), -1)

        x = torch.relu(self.fc1(x))
        x = self.fc2(x)

        return x

# Example usage:


NameError: ignored

In [ ]:
# Create a sample input tensor with shape (batch_size, input_size, sequence_length)
sample_input = torch.randn((1, 3, 128))  # Example input with 3 input channels and a sequence of length 128

# Instantiate the Discriminator with the number of classes for binary classification (e.g., 1 for real, 0 for fake)
num_classes = 1
discriminator = Discriminator(input_size=3, num_classes=num_classes)

# Forward pass to get the output
output = discriminator(sample_input)

print("Input shape:", sample_input.shape)
print("Output shape:", output.shape) 94215

In [ ]:
D = nn.Sequential(
    nn.Conv1d(205, 32, 3, stride=2),
    nn.Conv1d(32, 32, 3, stride=2),
    nn.MaxPool2d(3, stride=2),
    nn.Flatten(),
    nn.Linear(94215, 256),
    nn.LeakyReLU(0.2),
    nn.Linear(256, 1),
    nn.Sigmoid())

In [ ]:
def count_parameters(model):
    train_para = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_para = sum(p.numel() for p in model.parameters())

    print("train para: ",train_para )
    print("total para: ",total_para )
    print("% fraction: " , train_para / total_para)

In [ ]:
count_parameters(D)

train para:  24142369
total para:  24142369
% fraction:  1.0


In [ ]:
24142369/(1000000000/2)

0.048284738

In [ ]:
24140833/(1000000000/2)

0.048281666

In [ ]:
hidden_size = 256

In [ ]:
D2 = nn.Sequential(
    nn.Linear(28*28, 256),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid())

In [ ]:
count_parameters(D2)

train para:  267009
total para:  267009
% fraction:  1.0


In [ ]:
267009/(1000000000/2)

0.000534018

ValueError: ignored

In [ ]:
d

tensor([[0.4796],
        [0.4800],
        [0.4738],
        [0.4835],
        [0.4920],
        [0.4925]], grad_fn=<SigmoidBackward0>)

In [ ]:
one = torch.ones(6, 1)

In [ ]:
a = torch.randn(6,205,50257)

In [ ]:
def give_input_shape(a):

    c1 = nn.Conv1d(205, 32, 3, stride=2)
    c2 = nn.Conv1d(32, 32, 3, stride=2)
    p1 = nn.MaxPool2d(3, stride=2)
    f = nn.Flatten()

    a = c1(a)
    a = c2(a)
    a = p1(a)
    a = f(a)

    return a[1]


In [ ]:
print(input.shape)

a = c1(input)

print(a.shape)

a = c2(a)

print(a.shape)

a = p1(a)

print(a.shape)

a = f(a)

print(a.shape)

a = l1(a)

print(a.shape)

a = l2(a)

print(a.shape)

a = l2(a)

print(a.shape)

a = s(a)

print(a.shape)

torch.Size([6, 32, 25128])
torch.Size([6, 32, 12563])
torch.Size([6, 15, 6281])
torch.Size([6, 94215])
torch.Size([6, 256])
torch.Size([6, 256])
torch.Size([6, 256])
torch.Size([6, 256])


In [ ]:
def Discriminator():

    D = nn.Sequential(
    nn.Conv1d(205, 32, 3, stride=2),
    nn.Conv1d(32, 32, 3, stride=2),
    nn.MaxPool2d(3, stride=2),
    nn.Flatten(),
    nn.Linear(94215, 256),
    nn.LeakyReLU(0.2),
    nn.Linear(256, 1),
    nn.Sigmoid())

    return D

In [ ]:

class DynamicCNN(nn.Module):
    def __init__(self, num_classes):
        super(DynamicCNN, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv1d(in_channels=3, out_channels=16, kernel_size=3, stride=2)
        self.conv2 = nn.Conv1d(in_channels=3, out_channels=16, kernel_size=3, stride=2)
        self.pool1 = nn.MaxPool2d(3, stride=2)

        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully connected layers
        self.fc1 = nn.Linear(64, num_classes)

        # Global average pooling
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)

        x = self.conv3(x)
        x = self.relu3(x)
        x = self.pool3(x)

        x = self.global_avg_pool(x)

        x = x.view(x.size(0), -1)  # Flatten

        x = self.fc1(x)

        return x

In [ ]:
class DynamicCNN(nn.Module):
    def __init__(self, num_classes):
        super(DynamicCNN, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=input_channels, out_channels=16, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully connected layers
        self.fc1 = nn.Linear(64, num_classes)

        # Global average pooling
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))

    def forward(self, x):
        x = self.conv1(x)
        print(x.shape)
        x = self.relu1(x)
        print(x.shape)
        x = self.pool1(x)
        print(x.shape)
        x = self.conv2(x)
        print(x.shape)
        x = self.relu2(x)
        print(x.shape)
        x = self.pool2(x)
        print(x.shape)
        x = self.conv3(x)
        print(x.shape)
        x = self.relu3(x)
        print(x.shape)
        x = self.pool3(x)
        print(x.shape)
        x = self.global_avg_pool(x)
        print(x.shape)
        x = x.view(x.size(0), -1)  # Flatten
        print(x.shape)
        x = self.fc1(x)
        print(x.shape)

        return x

# Example input shape (adjust according to your data)
batch_size = 32
input_channels = 128
input_height = 128
input_width = 128
input_data = torch.randn(batch_size, input_channels, input_height, input_width)

# Create an instance of the DynamicCNN model
num_classes = 10  # Number of classes for classification
model = DynamicCNN(num_classes)

# Forward pass through the model
output = model(input_data)

torch.Size([32, 16, 128, 128])
torch.Size([32, 16, 128, 128])
torch.Size([32, 16, 64, 64])
torch.Size([32, 32, 64, 64])
torch.Size([32, 32, 64, 64])
torch.Size([32, 32, 32, 32])
torch.Size([32, 64, 32, 32])
torch.Size([32, 64, 32, 32])
torch.Size([32, 64, 16, 16])
torch.Size([32, 64, 1, 1])
torch.Size([32, 64])
torch.Size([32, 10])


In [ ]:
model

DynamicCNN(
  (conv1): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3): ReLU()
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=64, out_features=10, bias=True)
  (global_avg_pool): AdaptiveAvgPool2d(output_size=(1, 1))
)